<h1> Projeto 2 de Ciências de Dados </h1>

<h2> Grupo 20 </h2>
    <br>
    <h4> Integrantes: 
    <br>
    <br>
    <ul>
    <li> Venâncio Freitas </li>
    <br>
    <li> Welington Rodrigues </li>
    <br>
    <li> Henrique Puppi </li>
    <br>
    <li> Gabriel Fonseca </li>
    <br>
    </ul>
    </h4>

## 1. Visão Geral do Projeto 

### Objetivo 

 <p> O Trabalho em questão, envolverá a análise de uma base de dados na qual temos dados sociais sobre pessoas em situação de rua, a partir disso, iremos traçar o perfil envolvendo determinada situação de um desabrigado.</p>

### Motivações 

<li> Tendo em vista que, segundo o IBGE, o número de pessoas em situação de rua aumenta constantemente no Estado de São Paulo entende-se pelo grupo que é muito relevante estudar o perfil dessas, para entender quais suas necessidades e motivações que são oferecidos para cada tipo de contrato, e, assim, direcionar estudos de  possíveis fatores socioeconômicos que levem à essa escolha</li>

<li> Esse tipo de entendimento é muito relevante para cada prestador de serviços da área da saúde, pois, a partir do estudo de cada contrato, pode ser feito um levantamento hopitalar das possíveis demandas hospitalares necessárias. </li>

### Base de Dados 

<p> A base de dados utilizada encontra-se na plataforma de DataSUS, baseado em um levantamento feito pela Agência Nacional da Saúde Suplementar, na seção de 'Saúde', do site citado, e pode ser acessado pelo link a seguir : <a href='https://dados.gov.br/dados/conjuntos-dados/produtos-e-prestadores-hospitalares'> Data.Gov</a>, onde as features estão explicadas no 'Dicionário de Dados', na aba recursos. Além disso, possui um arquivo Zip com a base de dados referente a mais de 10 Estados Brasileiras, o que permite a análise para grandes extensões geográficas. 

### Target 

<p> A variável Target de estudo do grupo será a coluna 'SEGMENTACAO_ASSISTENCIAL' , que informa o tipo de assitência que cada plano de saúde fornece, contendo os segmentos de assitência de cada plano de saúde, que são: 
<br>
<br>
<li> AMBULATORIAL (Ambulatorial, Ambulatorial + Odontológico) </li>
<li> AMB + HOSP C/ OBST (Ambulatorial + Hospitalar com obstetrícia, Ambulatorial + Hospitalar com obstetrícia + Odontológico, Referência) </li>
<li> AMB + HOSP S/ OBST (Ambulatorial + Hospitalar sem obstetrícia , Ambulatorial + Hospitalar sem obstetrícia + Odontológico) </li>
<li> HOSP C/ OBST (Hospitalar com obstetrícia, Hospitalar com obstetrícia + Odontológico) </li>
<li> HOSP S/ OBST (Hospitalar sem obstetrícia, Hospitalar sem obstetrícia + Odontológico)</li>
<li> ODONTOLÓGICO (Odontológico)</li>

In [1]:
import pandas as pd
pd.options.mode.chained_assignment = None
from math import*
import numpy as np

### 1. Limpeza da Base de Dados

In [2]:
df = pd.read_excel('dados.xlsx')

In [3]:
df.Cor_Raça_Etnia.value_counts()

Parda                10866
Sem identificação     8815
Branca                5958
Preta                 5478
Não respondeu          395
Indígena               186
Amarela                186
Name: Cor_Raça_Etnia, dtype: int64

### 2. Análise Exploratória